In [9]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 29.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [12]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 51.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 57.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    confusion_matrix, 
    classification_report, 
    roc_auc_score
)
import joblib  # For saving the model

def evaluate_fnr(y_true, y_pred):
    """
    Calculate the False Negative Rate (FNR).
    
    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred : array-like
        Predicted labels

    Returns:
    --------
    float: False Negative Rate
    """
    cm = confusion_matrix(y_true, y_pred)
    fn = cm[1, 0]  # False negatives
    tp = cm[1, 1]  # True positives
    return fn / (fn + tp) if (fn + tp) > 0 else 0.0

def advanced_logistic_regression_perturbed(X_train, y_train, X_test, y_test):
    """
    Advanced Logistic Regression with perturbation analysis and model evaluation.
    
    Parameters:
    -----------
    X_train : array-like
        Training data features
    y_train : array-like
        Training data labels
    X_test : array-like
        Test data features
    y_test : array-like
        Test data labels
    
    Returns:
    --------
    dict: A comprehensive dictionary of model performance metrics and insights
    """
    # Create preprocessing pipeline to handle missing values and scaling
    preprocessing_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),  # Replace NaNs with median of each column
        ('scaler', StandardScaler())  # Scale the features
    ])
    
    # Preprocess the training data (perturbed)
    X_train_cleaned = preprocessing_pipeline.fit_transform(X_train)
    y_train_cleaned = y_train

    # Preprocess the test data using the same transformations
    X_test_cleaned = preprocessing_pipeline.transform(X_test)

    # Full pipeline with logistic regression
    model = LogisticRegression(max_iter=5000, class_weight='balanced', random_state=42)
    
    # Fit the model on the preprocessed training data
    model.fit(X_train_cleaned, y_train_cleaned)
    
    # Predictions for Logistic Regression
    y_pred = model.predict(X_test_cleaned)
    y_proba = model.predict_proba(X_test_cleaned)[:, 1]  # Probabilities for the positive class

    # Performance metrics
    performance_metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='weighted'),
        'recall': recall_score(y_test, y_pred, average='weighted'),
        'f1_score': f1_score(y_test, y_pred, average='weighted'),
        'roc_auc': roc_auc_score(y_test, y_proba),  # Using probabilities for AUC
        'false_negative_rate': evaluate_fnr(y_test, y_pred)  # False Negative Rate
    }

    # Print False Negative Rate
    print("False Negative Rate:", performance_metrics['false_negative_rate'])

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig('confusion_matrix_perturbed.png')
    plt.close()

    # Cross-validation scores
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train_cleaned, y_train_cleaned, cv=cv, scoring='f1_weighted')

    # Additional insights
    performance_metrics.update({
        'cross_val_scores': cv_scores,
        'mean_cv_score': np.mean(cv_scores),
        'std_cv_score': np.std(cv_scores)
    })

    # Print detailed classification report
    print("\nDetailed Classification Report:")
    print(classification_report(y_test, y_pred))

    # Learning Curve
    train_sizes, train_scores, test_scores = learning_curve(
        model, X_train_cleaned, y_train_cleaned, 
        cv=cv, train_sizes=np.linspace(0.1, 1.0, 10), 
        scoring='f1_weighted'
    )

    # Visualization of Learning Curve
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, np.mean(train_scores, axis=1), label='Training score')
    plt.plot(train_sizes, np.mean(test_scores, axis=1), label='Cross-validation score')
    plt.title('Learning Curve')
    plt.xlabel('Training Examples')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.tight_layout()
    plt.savefig('learning_curve_perturbed.png')
    plt.close()

    # Save the model probabilities
    np.save('logreg_perturbed_probabilities.npy', y_proba)

    # Optionally, save the model itself
    joblib.dump(model, 'logreg_model_perturbed.pkl')
    
    return performance_metrics, y_proba

# Load your perturbed data
X_train_perturbed = np.load('X_train_balanced_pertubed.npy')
y_train_perturbed = np.load('y_train_balanced_pertubed.npy')
X_test_perturbed = np.load('X_test_pertubed.npy')
y_test_perturbed = np.load('y_test_pertubed.npy')

# Call the function with the perturbed data
results, probabilities = advanced_logistic_regression_perturbed(
    X_train_perturbed, y_train_perturbed, X_test_perturbed, y_test_perturbed
)

# Print the performance metrics
print("Performance Metrics:", results)

False Negative Rate: 0.16883116883116883

Detailed Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      0.90      0.94      8254
         1.0       0.13      0.83      0.23       154

    accuracy                           0.90      8408
   macro avg       0.56      0.86      0.58      8408
weighted avg       0.98      0.90      0.93      8408

Performance Metrics: {'accuracy': 0.8952188392007612, 'precision': 0.9806315418250473, 'recall': 0.8952188392007612, 'f1_score': 0.9306473629044654, 'roc_auc': np.float64(0.9462370074800412), 'false_negative_rate': np.float64(0.16883116883116883), 'cross_val_scores': array([0.89467051, 0.88695431, 0.88786137, 0.89050344, 0.89157023]), 'mean_cv_score': np.float64(0.890311974474478), 'std_cv_score': np.float64(0.0027530368782404347)}
